In [17]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://dl.dell.com/content/manual24574503-integrated-dell-remote-access-controller-9-user-s-guide.pdf?language=en-us")
data = loader.load()

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [19]:
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf"))

In [29]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("how to access FRU information?")

docs

[Document(page_content='To view the SFP Transceiver device information using RACADM, use the hwinventory  command.\nFor more information, see the iDRAC RACADM CLI Guide  available at https://www.dell.com/idracmanuals .\nTelemetry Streaming\nTelemetry enables users to collect and stream real-time device metrics, events, and data logs from a PowerEdge server to a\nsubscribed external client or server application. Using Telemetry, you can set the type and frequency of reports that needs to be\ngenerated.', metadata={'page': 207, 'source': 'https://dl.dell.com/content/manual24574503-integrated-dell-remote-access-controller-9-user-s-guide.pdf?language=en-us'}),
 Document(page_content='information about the properties that are displayed on the web interface, see the iDRAC Online Help .\nNOTE:  The Hardware Overview page displays data only for sensors present on your system.\nTable 17. Sensor information using web interface and RACADM \nView sensor information For Using web interface Using RA

In [30]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat = ChatOllama(model="llama3", base_url="http://10.0.0.147:11434", temperature=0)

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [31]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Answer the user's questions based on the below context:\n\n{context}")), MessagesPlaceholder(variable_name='messages')])
| ChatOllama(base_url='http://10.0.0.147:11434', model='llama3', temperature=0.0)
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [37]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("how to access FRU information?")

response = document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

In [41]:
print(response)
demo_ephemeral_chat_history.add_ai_message(response)

To access FRU (Field Replaceable Unit) information using RACADM, you can use the following command:

1. Open a command prompt or terminal window.
2. Navigate to the directory where the RACADM executable is located.
3. Run the following command:
```
racadm iwracsensor -getfruinfo
```
This command retrieves FRU information from the system and displays it in a table format.

Alternatively, you can also access FRU information using the iDRAC web interface:

1. Log in to the iDRAC web interface.
2. Click on the "System" tab.
3. Click on "Overview".
4. Scroll down to the "FRU Information" section.
5. Click on the "View FRU Information" button.

This will display a table with information about the FRUs (Field Replaceable Units) installed in your system, including their serial numbers, part numbers, and other relevant details.


In [42]:
demo_ephemeral_chat_history.add_user_message("tell me more")

response = document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

In [44]:
print(response)

Here are some additional details on how to access FRU (Field Replaceable Unit) information using RACADM:

**Using RACADM:**

1. Open a command prompt or terminal window.
2. Navigate to the directory where the RACADM executable is located.
3. Run the following command:
```
racadm iwracsensor -getfruinfo
```
This command retrieves FRU information from the system and displays it in a table format.

**Using iDRAC Web Interface:**

1. Log in to the iDRAC web interface.
2. Click on the "System" tab.
3. Click on "Overview".
4. Scroll down to the "FRU Information" section.
5. Click on the "View FRU Information" button.

This will display a table with information about the FRUs (Field Replaceable Units) installed in your system, including their serial numbers, part numbers, and other relevant details.

**Additional Tips:**

* Make sure you have the correct version of RACADM installed on your system.
* If you are using RACADM for the first time, you may need to configure it by running the `racad